In [1]:
!python3 -m pip install -r "../requirements.txt"

ERROR: Could not open requirements file: [Errno 2] No such file or directory: '../requirements.txt'


In [ ]:
!pip install sentence_transformers

In [ ]:
!pip install --upgrade transformers

In [3]:
# Import all libs
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sklearn.metrics import (
    f1_score, recall_score, balanced_accuracy_score,
    matthews_corrcoef, precision_score, average_precision_score
)
from sklearn.preprocessing import label_binarize
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

In [6]:
# Authenticate with HuggingFace
import os
from dotenv import load_dotenv
from huggingface_hub import login

from google.colab import userdata
hugging_face_token = userdata.get("hf_token") #If using gg colab

# load_dotenv() #If using VSCode
# hugging_face_token = os.getenv("hf_token") #If using VSCode

login(token=hugging_face_token)

In [7]:
# Check memory allocated for MPS
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("MPS device available:", torch.backends.mps.is_available())
    # This tells you memory usage is not directly exposed, but you can monitor allocated tensors
    x = torch.randn(1024, 1024, device=device)
    print("Tensor allocated on MPS:", x.device)
else:
    print(f"CUDA device available: {torch.cuda.is_available()}")

CUDA device available: True


Load AG news datasets

In [10]:
label_map = {
    0: "world",
    1: "sports",
    2: "business",
    3: "sci/tech"
}
# Load existing prepared files (kept for reference)
ag_news_imbalanced_data_99_to_1 = pd.read_parquet("Data/ag_news/ag_news_train_imbalanced_99_to_1.parquet")
balanced_data = pd.read_parquet("Data/ag_news/ag_news_train_balanced.parquet")
ag_news_imbalanced_data_49_to_1 = pd.read_parquet("Data/ag_news/ag_news_train_imbalanced_49_to_1_ratio.parquet")

# Map numeric labels into text labels
balanced_data["label"] = balanced_data["label"].map(label_map)
ag_news_imbalanced_data_99_to_1["label"] = ag_news_imbalanced_data_99_to_1["label"].map(label_map)
ag_news_imbalanced_data_49_to_1["label"] = ag_news_imbalanced_data_49_to_1["label"].map(label_map)

# Shuffle the dataset
ag_news_imbalanced_data_99_to_1 = ag_news_imbalanced_data_99_to_1.sample(frac=1).reset_index(drop=True)
balanced_data = balanced_data.sample(frac=1).reset_index(drop=True)
ag_news_imbalanced_data_49_to_1 = ag_news_imbalanced_data_49_to_1.sample(frac=1).reset_index(drop=True)

# Utility: create imbalanced AG News dataset programmatically by choosing a majority label
def split_ratio_for_ag_news(df, majority_label, majority_count, minority_count):
    """Create an imbalanced AG News subset where `majority_label` has `majority_count` samples"""
    parts = []
    labels = df['label'].unique().tolist()
    for lab in labels:
        if lab == majority_label:
            parts.append(df[df['label'] == lab].sample(majority_count, random_state=42))
        else:
            # sample minority_count from each other class
            parts.append(df[df['label'] == lab].sample(minority_count, random_state=42))
    out = pd.concat(parts, ignore_index=True, sort=False)
    return out.sample(frac=1).reset_index(drop=True)


ag_news_world_majority_99 = split_ratio_for_ag_news(balanced_data, 'world', majority_count=980, minority_count=20)
ag_news_sports_majority_99 = split_ratio_for_ag_news(balanced_data, 'sports', majority_count=980, minority_count=20)
ag_news_business_majority_99 = split_ratio_for_ag_news(balanced_data, 'business', majority_count=980, minority_count=20)

# Keep original balanced_data as the balanced dataset
# Testing
balanced_data


,text,label
0,"Legal Cloud Lingers for Rosa Parks Rosa Parks,...",world
1,SIX OF THE BEST FOR MILAN Reigning Italian cha...,sports
2,Heartbeat away from third term TONY Blair yest...,world
3,Howard meets Aboriginal star Aboriginal sports...,world
4,"Illinois, Michigan both want new federal isoto...",sci/tech
...,...,...
3995,Giants Shift Blame from Coach to the Players T...,sports
3996,German welfare rallies escalate Eastern German...,business
3997,United Says It Needs \$2 Bln More in Cost Cuts...,business
3998,What #39;s gone wrong with Tiger #39;s swing -...,sports


Load toxic text dataset (The dataset is already imbalanced)

- Rename the columns to have the same names as other datasets
- Label = 0 --> Not toxic
- Label = 1 --> Toxic
- Map label column into words

In [11]:
toxic_label_map = {
    0: "nontoxic",
    1: "toxic"
}

def split_ratio_for_toxic_dataset(df, majority_label='nontoxic', majority_count=500, minority_count=20):
    """Create an imbalanced toxic_text subset where `majority_label` has `majority_count` samples and the other label has `minority_count`."""
    parts = []
    for lab in df['label'].unique():
        if lab == majority_label:
            parts.append(df[df['label'] == lab].sample(majority_count, random_state=42))
        else:
            parts.append(df[df['label'] == lab].sample(minority_count, random_state=42))
    out = pd.concat(parts, ignore_index=True, sort=False)
    return out.sample(frac=1).reset_index(drop=True)

toxic_text = pd.read_csv("Data/toxic_text/train.csv")
toxic_text = toxic_text[["comment_text", "toxic"]]
toxic_text = toxic_text.rename(columns={"comment_text": "text", "toxic": "label"})
toxic_text["label"] = toxic_text["label"].map(toxic_label_map)

# Get 3 small subsets of the main datasets with 3 different ratios and different majority classes
toxic_balanced = split_ratio_for_toxic_dataset(toxic_text, majority_label='nontoxic', majority_count=500, minority_count=500)
# Increase minority_count to 20 as requested
toxic_99_to_1 = split_ratio_for_toxic_dataset(toxic_text, majority_label='nontoxic', majority_count=980, minority_count=20)
toxic_49_to_1 = split_ratio_for_toxic_dataset(toxic_text, majority_label='nontoxic', majority_count=940, minority_count=20)
toxic_toxic_majority_99 = split_ratio_for_toxic_dataset(toxic_text, majority_label='toxic', majority_count=980, minority_count=20)


Load twitter emotion type dataset (This is also imbalanced)
- Create 3 small datasets, roughly 2000 rows each, with balanced, 99:1, 49:1 imbalanced ratios

In [13]:
emotion_map = {
    0: "sadness",
    1: "joy",
    2: "love",
    3: "anger",
    4: "fear",
    5: "surprise"
}
emotion_df = pd.read_parquet("Data/twit/twitter_emotion.parquet")
emotion_df["label"] = emotion_df["label"].map(emotion_map)

def split_ratio_for_emotion_dataset(df, majority_label='sadness', majority_count=200, minority_count=20):
    """Create an imbalanced emotion subset where `majority_label` has `majority_count` samples and every other label has `minority_count`."""
    parts = []
    labels = df['label'].unique().tolist()
    for lab in labels:
        if lab == majority_label:
            parts.append(df[df['label'] == lab].sample(majority_count, random_state=42))
        else:
            parts.append(df[df['label'] == lab].sample(minority_count, random_state=42))
    out = pd.concat(parts, ignore_index=True, sort=False)
    return out.sample(frac=1).reset_index(drop=True)

# Get 3 small subsets of the main datasets with 3 different ratios and different majority labels
emotion_balanced = split_ratio_for_emotion_dataset(emotion_df, majority_label='sadness', majority_count=200, minority_count=200)
# Use minority_count=20 as requested to reduce random variation
emotion_imbalanced_99_to_1 = split_ratio_for_emotion_dataset(emotion_df, majority_label='sadness', majority_count=950, minority_count=20)
emotion_imbalanced_49_to_1 = split_ratio_for_emotion_dataset(emotion_df, majority_label='sadness', majority_count=202, minority_count=20)
# Also create variants where the majority class is 'joy' or others to compare
emotion_joy_majority_99 = split_ratio_for_emotion_dataset(emotion_df, majority_label='joy', majority_count=950, minority_count=20)
emotion_love_majority_99 = split_ratio_for_emotion_dataset(emotion_df, majority_label='love', majority_count=950, minority_count=20)

# Quick check
emotion_imbalanced_99_to_1["label"].value_counts()




,count
label,
sadness,950
love,20
joy,20
fear,20
anger,20
surprise,20


Function to build instruction for the LLMs, which can be fit with all 3 classification datasets

In [14]:
import random
def build_prompt(df, text, label_map, shots_per_class=None):
    """
    Function to construct an instruction for the LLM

    Args:
        text (str): The text of the data

    Returns:
        prompt (str): The constructed prompt for the LLM
    """
    assert shots_per_class is not None, "Please provide 'shots_per_class' parameter"
    prompt = (
        f"You are a powerful, precise, and helpful assistant that classifies text into well-defined categories, NO MATTER THE CONTEXT."
        f" IMPORTANT: CHOOSE ONE WORD FROM THESE CATEGORIES: {', '.join(list(label_map.values()))}."
        f" Respond with exactly one word: the single best category inside the given categories, DO NOT ANSWER ANY OTHER CATEGORIES BESIDES THE GIVEN ONE."
        f" Do not explain your choice, provide reasoning, or output anything else."

    )
    # ASSUME THE shots_per_class WILL ALWAYS BE PASSED
    few_shots_example = []
    for lab in list(label_map.values()):
        samples = df[df['label'] == lab].sample(shots_per_class, random_state=42)
        for _, r in samples.iterrows():
            few_shots_example.append({'text': r['text'],
                                      'label': r["label"]})

    # Shuffle the shots examples
    random.shuffle(few_shots_example)

    # If shots_per_class = 0, then dont provide anything else but the prompt
    if shots_per_class != 0:
        prompt += f" Learn from these examples to understand context and edge cases: "
        prompt += "\n\n"
        for ex in few_shots_example:
            # print(ex)
            prompt += f"Review: \"{ex['text']}\"\nCategory: {ex['label']}\n\n"
        prompt += f"Review: \"{text}\"\nCategory:" #Leave Category here blank since we want the LLM to generate text
    return prompt


# Testing function
print(build_prompt(ag_news_imbalanced_data_99_to_1, "Astros Rally Past the Giants With one swing of the bat, Lance Berkman revived the Houston Astros' playoff hopes - and " \
"gave the Los Angeles Dodgers a much-needed reprieve. Berkman hit a three-run homer " \
"off Dustin Hermanson, highlighting a five-run ninth inning that sent Houston to a 7-3 win over San Francisco on Thursday night...", label_map, shots_per_class=3))

You are a powerful, precise, and helpful assistant that classifies text into well-defined categories, NO MATTER THE CONTEXT. IMPORTANT: CHOOSE ONE WORD FROM THESE CATEGORIES: world, sports, business, sci/tech. Respond with exactly one word: the single best category inside the given categories, DO NOT ANSWER ANY OTHER CATEGORIES BESIDES THE GIVEN ONE. Do not explain your choice, provide reasoning, or output anything else. Learn from these examples to understand context and edge cases: 

Review: "Right time for new A #39;s hero McMillon #39;s homer in ninth breaks tie &lt;b&gt;...&lt;/b&gt; He was thinking about that, the little-used outfielder admitted, when he came up with two on and one out in the bottom of the ninth, the game tied 6-6."
Category: sports

Review: "Merck to Withdraw Vioxx Because of Heart Risks (Update4) Merck  amp; Co. withdrew its Vioxx painkiller, which generated \$2.5 billion in sales last year, because of a link to heart attacks and strokes."
Category: business

R

In [15]:
def clean_time(time):
  """
  Function to clean the time into prettier format, returns the better format of time
  """
  if time <= 60:
    return f"{time} seconds."

  minutes = time // 60
  remain_sec = time - minutes * 60
  return f"{minutes} minutes, {remain_sec:.2f} seconds."


Label normalization using semantic similarity
- Since we have 3 datasets, using manual variation map will not ensure every predictions that the LLM makes
- So we load a sentence embedding model to calculate the nearest vector amongst the label using Cosine Similarity

In [16]:
from sentence_transformers import SentenceTransformer, util

# Testing with AG News

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Ag news
valid_labs_ag_news = list(label_map.values())
valid_embeddings_ag_news = embedding_model.encode(valid_labs_ag_news, convert_to_tensor=True)

# Toxic Text
valid_labs_toxic_text = list(toxic_label_map.values())
valid_embeddings_toxic_text = embedding_model.encode(valid_labs_toxic_text, convert_to_tensor=True)

# Twitter Emotion
valid_labs_emotion = list(emotion_map.values())
valid_embeddings_emotion = embedding_model.encode(valid_labs_emotion, convert_to_tensor=True)

def normalize(label, valid_embeddings, valid_labs):
    pred_emb = embedding_model.encode(label, convert_to_tensor=True)
    cos_scores = util.cos_sim(pred_emb, valid_embeddings)[0]
    closest_idx = cos_scores.argmax().item()
    return valid_labs[closest_idx]

# Testing for AG News
raw_preds = "I am so sad"
normalized_preds = normalize(raw_preds, valid_embeddings=valid_embeddings_emotion, valid_labs=valid_labs_emotion)
print(normalized_preds)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

sadness


In [17]:

from tqdm.auto import tqdm
from transformers import pipeline, logging
from time import time


# Load model


# CREATE A FUNCTION TO RUN CLASSFICATION
def classify(model, df, label_map, shots, batch_size=16, max_new_tokens=3, valid_embeddings=None, valid_labs=None):
    """
    Function to run classification with different number of shots

    Args:
        model (str): name of the model
        tokenizer
        df (pd.DataFrame): the pandas dataframe
        batch_size (int): batch size per run

    Returns:
        pred_arr (List[str]): the array that contains all predictions
    """
    # Initiate a pipeline for each dataset
    # USE text2text-generation for the gemma model
    # USE text-generation for the others, or text-classification
    # USE fill-mask for distillbert
    pipe = pipeline("text-generation", model=model, dtype=torch.float16)
    logging.set_verbosity_error()

    # Generate prompts for all rows
    prompts = [build_prompt(df, text, label_map, shots_per_class=shots) for text in df["text"]]

    # Run the pipeline for each row
    pred_arr = []
    start_time = time()

    for i in range(0, len(prompts), batch_size):
        batch = prompts[i:i + batch_size] #slices a sublist of prompts
        results = pipe(batch, max_new_tokens=max_new_tokens, do_sample=False)
        for prompt, res in zip(batch, results):
            pred = res[0]['generated_text'][len(prompt):].strip().lower().split()
            # print(f"Real value: {df["label"]}")
            # print(f"Predicted value: {pred}")
            if pred[0] not in list(label_map.values()):
                normalized_pred = normalize(pred[0], valid_embeddings=valid_embeddings, valid_labs=valid_labs)
                pred_arr.append(normalized_pred)
            else:
                pred_arr.append(pred[0]) #Use pred[0] for some cases
    end_time = time()

    total_time = clean_time(end_time - start_time)

    print("Total running time is " + total_time)
    return pred_arr



Function to compute all metrics

In [18]:
# Pass list(df["text"]) for y_true
# list(label_map.values())

def eval_llm(y_true, y_pred, label_map):
    y_true_arr = np.array([x.lower().strip() for x in y_true])
    # print(y_pred)  # avoid noisy output
    y_pred_arr = np.array([x.lower().strip() for x in y_pred])

    labels = [lab.lower() for lab in list(label_map.values())]

    # Calculate macro scores:
    macro_f1 = f1_score(y_true_arr, y_pred_arr, labels=labels, zero_division=0, average='macro')
    macro_recall = recall_score(y_true_arr, y_pred_arr, labels=labels, average='macro', zero_division=0)
    bal_acc = balanced_accuracy_score(y_true_arr, y_pred_arr)
    mcc = matthews_corrcoef(y_true_arr, y_pred_arr)

    # Calculate per-class precision, recall, f1 (returns arrays aligned with labels)
    precision_per_class_vals = precision_score(y_true_arr, y_pred_arr, labels=labels, average=None, zero_division=0)
    recall_per_class_vals = recall_score(y_true_arr, y_pred_arr, labels=labels, average=None, zero_division=0)
    f1_per_class_vals = f1_score(y_true_arr, y_pred_arr, labels=labels, average=None, zero_division=0)

    precision_per_class = {}
    recall_per_class = {}
    f1_per_class = {}
    for idx, cls in enumerate(labels):
        precision_per_class[cls] = float(precision_per_class_vals[idx])
        recall_per_class[cls] = float(recall_per_class_vals[idx])
        f1_per_class[cls] = float(f1_per_class_vals[idx])

    # Calculate AUPRC per class
    y_true_bin = label_binarize(y_true_arr, classes=labels)
    y_pred_bin = label_binarize(y_pred_arr, classes=labels)
    if len(labels) == 2 and y_true_bin.shape[1] == 1:
        y_true_bin = np.hstack([1 - y_true_bin, y_true_bin])
        y_pred_bin = np.hstack([1 - y_pred_bin, y_pred_bin])

    auprc_per_class = {}
    for idx, cls in enumerate(labels):
        ap = average_precision_score(y_true_bin[:, idx], y_pred_bin[:, idx])
        auprc_per_class[cls] = float(ap)

    return {
        "macro_f1": float(macro_f1),
        "macro_recall": float(macro_recall),
        "balanced_accuracy": float(bal_acc),
        "mcc": float(mcc),
        "auprc_per_class": auprc_per_class,
        "precision_per_class": precision_per_class,
        "recall_per_class": recall_per_class,
        "f1_per_class": f1_per_class
    }



NOW FOCUSING ON QWEN2.5 INSTRUCT MODEDLS

In [22]:
# model, df, label_map, shots, batch_size=8, max_new_tokens=3
def run_experiments(model, datasets_dict, dataset_name, label_map, shots_list=[2,4,8], batch_size=16, valid_embeddings=None, valid_labs=None):
    import os
    import re
    from datetime import datetime
    import numpy as _np
    import pandas as _pd

    def infer_meta(ds_name, df):
        # Try to infer ratio and majority label from the dataset name using heuristics
        ratio = 'unknown'
        maj_label = 'unknown'

        # Check common name patterns
        if 'balanced' in ds_name:
            ratio = 'balanced'
        m = re.search(r"(\d+)_to_(\d+)", ds_name)
        if m:
            ratio = f"{m.group(1)}:{m.group(2)}"

        # look for "_majority_" pattern like 'world_majority_99'
        m2 = re.search(r"([A-Za-z0-9]+)_majority", ds_name)
        if m2:
            maj_label = m2.group(1)

        # fallback: infer from df counts
        try:
            counts = df['label'].value_counts()
            if len(counts) > 0:
                maj_label_from_df = counts.idxmax()
                if maj_label == 'unknown':
                    maj_label = str(maj_label_from_df)
                # compute numeric ratio using most common and the minimum of others
                maj_count = int(counts.max())
                others = counts.drop(maj_label_from_df)
                if len(others) > 0:
                    min_count = int(others.min())
                else:
                    min_count = 0
                ratio = f"{maj_count}:{min_count}"
        except Exception:
            # keep heuristics result
            pass

        return ratio, maj_label

    results = []
    # Ensure results folder exists for this dataset
    out_dir = os.path.join("results", dataset_name)
    os.makedirs(out_dir, exist_ok=True)

    for ds_name, df in datasets_dict.items():
        print(f"=== RUNNING DATASET {ds_name} ===")
        test_df = df.sample(frac=1).reset_index(drop=True)
        for shots in shots_list:
            print(f"    === SHOTS = {shots} ===")
            preds = classify(model, test_df, label_map, shots=shots, batch_size=batch_size, valid_embeddings=valid_embeddings, valid_labs=valid_labs)
            metrics = eval_llm(test_df['label'].tolist(), preds, label_map=label_map)

            # infer dataset metadata
            ratio, maj_label = infer_meta(ds_name, df)

            row = {
                "model": model,
                "dataset": ds_name,
                "shots": shots,
                "dataset_ratio": ratio,
                "majority_label": maj_label,
                **metrics
            }
            results.append(row)

            # Create a flattened aggregated DataFrame for better CSV readability
            df_agg = _pd.json_normalize(results)
            # replace dots in column names (from nested dicts) with underscores
            df_agg.columns = [c.replace('.', '_') for c in df_agg.columns]


            agg_name = f"few_shot_results_{model.replace('/','_')}.csv"
            agg_path = os.path.join(out_dir, agg_name)
            df_agg.to_csv(agg_path, index=False)

            # New per-params file: group by (model, ds_name, ratio, maj_label, shots)
            safe_model = model.replace('/', '_')
            safe_ds_name = ds_name.replace(' ', '_')
            safe_maj = str(maj_label).replace(' ', '_').replace('/', '_')
            ratio_safe = str(ratio).replace(':', '-')
            params_fname = f"results__{safe_model}__{safe_ds_name}__ratio-{ratio_safe}__majority-{safe_maj}__shots-{shots}.csv"
            params_path = os.path.join(out_dir, params_fname)

            # Flatten the current row and append to the per-params CSV (create if not exists)
            flat_row_df = _pd.json_normalize([row])
            flat_row_df.columns = [c.replace('.', '_') for c in flat_row_df.columns]

            if _pd.io.common.file_exists(params_path):
                # append without header
                flat_row_df.to_csv(params_path, mode='a', header=False, index=False)
            else:
                flat_row_df.to_csv(params_path, index=False)

    return _pd.DataFrame(results)


In [20]:
# Create a dataset dict for easy mapping

ag_news_datasets_dict = {
    "ag_news_balanced": balanced_data,
    "ag_news_imbalanced_data_99_to_1": ag_news_imbalanced_data_99_to_1,
    "ag_news_imbalanced_data_49_to_1": ag_news_imbalanced_data_49_to_1,
    # Variants where a specific class is the majority (minority_count=20)
    "ag_news_world_majority_99": ag_news_world_majority_99,
    "ag_news_sports_majority_99": ag_news_sports_majority_99,
    "ag_news_business_majority_99": ag_news_business_majority_99
}

toxic_datasets_dict = {
    "toxic_text": toxic_balanced,
    "toxic_99_to_1": toxic_99_to_1,
    "toxic_49_to_1": toxic_49_to_1,
    "toxic_toxic_majority_99": toxic_toxic_majority_99
}

emotion_datasets_dict = {
    "emotion_df": emotion_balanced,
    "emotion_imbalanced_99_to_1": emotion_imbalanced_99_to_1,
    "emotion_imbalanced_49_to_1": emotion_imbalanced_49_to_1,
    "emotion_joy_majority_99": emotion_joy_majority_99,
    "emotion_love_majority_99": emotion_love_majority_99
}

Run models + evals

In [21]:
model = "Qwen/Qwen2.5-1.5B-Instruct"

Quantize model (Optional)
- Use subprocess library to run shell script

In [ ]:
from pathlib import Path
script_path = Path("../scripts/convert.py")
print("cwd:", Path.cwd())

cwd: /Volumes/huysuy05/Projects/Bias_of_LLMs/src


In [ ]:
import subprocess

cmd = ["python", str(script_path), "--hf-path", model]

try:
    res = subprocess.run(cmd, capture_output=True, text=True, check=True)
    print("STDOUT: ", res.stdout)
    print("STDERR: ", res.stderr)

except subprocess.CalledProcessError as e:
    print(f"Script failed with error: {e}")
    print("STDOUT:", e.stdout)
    print("STDERR:", e.stderr)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


STDOUT:  [INFO] Loading

STDERR:  
Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 115971.54it/s]



In [23]:
# RUN AG NEWS DATASET

res_df = run_experiments(model, ag_news_datasets_dict, 'ag_news',label_map, shots_list=[0,2,4,8], valid_embeddings=valid_embeddings_ag_news, valid_labs=valid_labs_ag_news)

=== RUNNING DATASET ag_news_balanced ===
    === SHOTS = 0 ===


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


Total running time is 4.0 minutes, 21.55 seconds.
    === SHOTS = 2 ===
Total running time is 6.0 minutes, 46.68 seconds.
    === SHOTS = 4 ===
Total running time is 6.0 minutes, 52.92 seconds.
    === SHOTS = 8 ===
Total running time is 7.0 minutes, 40.63 seconds.
=== RUNNING DATASET ag_news_imbalanced_data_99_to_1 ===
    === SHOTS = 0 ===
Total running time is 4.0 minutes, 24.34 seconds.
    === SHOTS = 2 ===
Total running time is 6.0 minutes, 45.68 seconds.
    === SHOTS = 4 ===
Total running time is 6.0 minutes, 50.84 seconds.
    === SHOTS = 8 ===
Total running time is 7.0 minutes, 41.18 seconds.
=== RUNNING DATASET ag_news_imbalanced_data_49_to_1 ===
    === SHOTS = 0 ===
Total running time is 4.0 minutes, 27.09 seconds.
    === SHOTS = 2 ===
Total running time is 6.0 minutes, 49.15 seconds.
    === SHOTS = 4 ===
Total running time is 6.0 minutes, 57.14 seconds.
    === SHOTS = 8 ===
Total running time is 7.0 minutes, 54.66 seconds.
=== RUNNING DATASET ag_news_world_majority_99 

In [24]:
# RUN TOXIC TEXT DATASET
res_df = run_experiments(model, toxic_datasets_dict, "toxic_text",toxic_label_map, shots_list=[0,2,4,8], valid_embeddings=valid_embeddings_toxic_text, valid_labs=valid_labs_toxic_text)

=== RUNNING DATASET toxic_text ===
    === SHOTS = 0 ===
Total running time is 1.0 minutes, 44.56 seconds.
    === SHOTS = 2 ===
Total running time is 1.0 minutes, 59.58 seconds.
    === SHOTS = 4 ===
Total running time is 2.0 minutes, 9.09 seconds.
    === SHOTS = 8 ===
Total running time is 2.0 minutes, 18.20 seconds.
=== RUNNING DATASET toxic_99_to_1 ===
    === SHOTS = 0 ===
Total running time is 1.0 minutes, 44.71 seconds.
    === SHOTS = 2 ===
Total running time is 1.0 minutes, 41.51 seconds.
    === SHOTS = 4 ===
Total running time is 1.0 minutes, 42.27 seconds.
    === SHOTS = 8 ===
Total running time is 1.0 minutes, 46.90 seconds.
=== RUNNING DATASET toxic_49_to_1 ===
    === SHOTS = 0 ===
Total running time is 1.0 minutes, 40.15 seconds.
    === SHOTS = 2 ===
Total running time is 1.0 minutes, 36.56 seconds.
    === SHOTS = 4 ===
Total running time is 1.0 minutes, 38.76 seconds.
    === SHOTS = 8 ===
Total running time is 1.0 minutes, 40.88 seconds.
=== RUNNING DATASET toxic_

In [25]:
# RUN TWITTER EMOTION DATASET
res_df = run_experiments(model, emotion_datasets_dict, 'twitter_emotion', emotion_map, shots_list=[0,2,4,8], valid_embeddings=valid_embeddings_emotion, valid_labs=valid_labs_emotion)

=== RUNNING DATASET emotion_df ===
    === SHOTS = 0 ===
Total running time is 1.0 minutes, 58.09 seconds.
    === SHOTS = 2 ===
Total running time is 2.0 minutes, 0.57 seconds.
    === SHOTS = 4 ===
Total running time is 2.0 minutes, 3.78 seconds.
    === SHOTS = 8 ===
Total running time is 2.0 minutes, 10.27 seconds.
=== RUNNING DATASET emotion_imbalanced_99_to_1 ===
    === SHOTS = 0 ===
Total running time is 1.0 minutes, 42.89 seconds.
    === SHOTS = 2 ===
Total running time is 1.0 minutes, 45.33 seconds.
    === SHOTS = 4 ===
Total running time is 1.0 minutes, 47.51 seconds.
    === SHOTS = 8 ===
Total running time is 1.0 minutes, 50.91 seconds.
=== RUNNING DATASET emotion_imbalanced_49_to_1 ===
    === SHOTS = 0 ===
Total running time is 29.52052927017212 seconds.
    === SHOTS = 2 ===
Total running time is 30.691033840179443 seconds.
    === SHOTS = 4 ===
Total running time is 31.037949800491333 seconds.
    === SHOTS = 8 ===
Total running time is 32.319969177246094 seconds.
==